In [1]:
import torch

import argparse
import dill
import glob
import importlib
import os
import pandas as pd
from tqdm import tqdm

from criterion import Criterion
from data import *
from dataset import *
from test import initilize_testing_loader, evaluate
from utils import parse_config, set_seeds, set_devices, load_pretrained_weight, load_checkpoint

In [2]:
parser = argparse.ArgumentParser(description='Visualizing')
parser.add_argument('--trial', default='shapenet_with_norm.rscnn.0001', type=str,
                    help='The trial name with its version, e.g., modelnet40.dgcnn.0001.')
parser.add_argument('--show_configs', default=False, type=bool,
                    help='Whether to print the config at the program beginning.')
parser.add_argument('--show_loss_details', default=False, type=bool,
                    help='Whether to show all loss details.')
parser.add_argument('--separator_bar', default='*' * 100, type=str,
                    help='Separator bar.')
parser.add_argument('--max_float_digits', default=4, type=int,
                    help='Limit the max digits to display.')
parser.add_argument('--pretrained_weight_path', default='', type=str,
                    help='The pre-trained weight path.')
parser.add_argument('--checkpoint_path', default='best', type=str,
                    help='Checkpoint path. "best" means the best checkpoint of saved ones.')

_StoreAction(option_strings=['--checkpoint_path'], dest='checkpoint_path', nargs=None, const=None, default='best', type=<class 'str'>, choices=None, help='Checkpoint path. "best" means the best checkpoint of saved ones.', metavar=None)

In [3]:
args = parser.parse_args(args=[])
parse_config(args)

    
# Dynamically import the model class, Net.
model_path = os.path.join(args.trial_dir, 'model')
model_package = importlib.import_module(model_path.replace('/', '.'))

# Set the environment.
set_seeds(args.seed)
device = set_devices(args.cuda_devices, args.separator_bar)

# Load the dataset.
loader = initilize_testing_loader(args)

# Set the model.
model = model_package.Net(loader.dataset.num_classes(args.task_type),
                            **args.net_arguments).to(device)
load_pretrained_weight(args.pretrained_weight_path, model, device)
load_checkpoint(args.trial_dir, args.checkpoint_path, model, device)

criterion = Criterion(args.task_type,
                          args.criterion,
                          args.show_loss_details,
                          args.max_float_digits).to(device)

criterion.reset()
base_results = evaluate(args, loader, model, device, criterion)
base_detils = criterion.all_sample_details

Running on cuda device: Tesla M40 24GB 

****************************************************************************************************

Load the best checkpoint: trial/shapenet_with_norm/rscnn/0001/checkpoints/epoch_42_instance-miou_0.8460.weight


Testing: 100%|██████████| 103/103 [01:57<00:00,  1.14s/it, class miou=0.8138, instance miou=0.8450, mean accuracy=0.9386, mean loss=0.1952]


In [4]:
if args.task_type == 'segmentation':
    for y, ious in criterion.global_dict['y_to_ious'].items():
        iou_sum = sum(ious)
        base_results[loader.dataset.categories[y]] = iou_sum / len(ious)

In [5]:
comparion_results = {'base' : base_results}

pd.DataFrame(comparion_results)

,base
Airplane,0.823082
Bag,0.782171
Cap,0.827233
Car,0.770402
Chair,0.900588
Earphone,0.763456
Guitar,0.910805
Knife,0.853467
Lamp,0.831992
Laptop,0.946519


In [8]:
pd.DataFrame(base_detils).T

,miou,category
2264,0.969867,Table
1837,0.979392,Laptop
2290,0.623915,Table
900,0.974032,Chair
2181,0.983069,Table
...,...,...
2718,0.951389,Table
1611,0.857276,Lamp
832,0.946364,Chair
824,0.955572,Chair


In [4]:
%tb

No traceback available to show.
